In [43]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

sdir = './scrub/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
data = pd.read_csv(f'multilabel_v1.csv')[['content', 'target']].sample(frac=1)
data['target'] = data['target'].apply(lambda x: eval(x))

In [45]:
t = pd.DataFrame(data['target'].values.tolist(), columns = [f'cls_{i+1}' for i in range(15)])
t.sum()

cls_1     439.0
cls_2     212.0
cls_3     249.0
cls_4     624.0
cls_5     185.0
cls_6     289.0
cls_7     417.0
cls_8     529.0
cls_9     470.0
cls_10    252.0
cls_11    857.0
cls_12    121.0
cls_13    253.0
cls_14    148.0
cls_15    157.0
dtype: float64

In [46]:
t = pd.read_excel(f'{sdir}t4m_2022_05_05__21_59.xlsx')
cnames = list(set(t['class_name'].values))
idx2cls = {i:v for i,v in enumerate(cnames)}
cls2idx = {v:i for i,v in enumerate(cnames)}
display(idx2cls)
cls2idx

{0: 'цена',
 1: 'оплата',
 2: 'лояльность',
 3: 'регистрация/коды',
 4: 'купоны',
 5: 'обслуживание',
 6: 'uxui',
 7: 'глюки_баги_тормоза',
 8: 'другое',
 9: 'обновление',
 10: 'доставка_общее',
 11: 'аккаунт',
 12: 'долгое_ожидание_доставки',
 13: 'создание_заказа',
 14: 'не_возвращаются_деньги_отмененного_заказа'}

{'цена': 0,
 'оплата': 1,
 'лояльность': 2,
 'регистрация/коды': 3,
 'купоны': 4,
 'обслуживание': 5,
 'uxui': 6,
 'глюки_баги_тормоза': 7,
 'другое': 8,
 'обновление': 9,
 'доставка_общее': 10,
 'аккаунт': 11,
 'долгое_ожидание_доставки': 12,
 'создание_заказа': 13,
 'не_возвращаются_деньги_отмененного_заказа': 14}

In [47]:
from datasets import load_dataset,Dataset,DatasetDict, load_metric

from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn


In [48]:
data = data[['content', 'target']].set_axis(['headline', 'label'], axis=1)
data=Dataset.from_pandas(data)

# 80% train, 20% test + validation
train_testvalid = data.train_test_split(test_size=0.3,seed=15)

# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5,seed=15)

# gather everything to have a single DatasetDict
data = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train'],
                   })

data['train'][0]

{'headline': 'Ужасно неудобное приложение',
 'label': [0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 '__index_level_0__': 895}

In [49]:
BATCH_SIZE = 8
seq_len = 512
n_classes = len(idx2cls.items())
out_features=768

# device = torch.device('mps')
# couldn't make distilbert uncased work on apple silicon, so i took random bert trained on news.
# also found bert trained on rotten tomatoes, but it was big and slow
checkpoint = 'cointegrated/rubert-tiny2'
# checkpoint = 'cointegrated/rubert-tiny'
checkpoint = 'DeepPavlov/distilrubert-small-cased-conversational'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.model_max_len=seq_len


def tokenize(batch):
    return tokenizer(batch["headline"], truncation=True,max_length=seq_len)

tokenized_dataset = data.map(tokenize, batched=True)

tokenized_dataset.set_format("torch",columns=["input_ids", "attention_mask", "label"])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


class CustomModel(nn.Module):
    def __init__(self, checkpoint, num_labels, out_features=768):
        super(CustomModel,self).__init__()
        self.num_labels = num_labels

        #Load Model with given checkpoint and extract its body
        self.model = AutoModel.from_pretrained(checkpoint,
                                                config=AutoConfig.from_pretrained(checkpoint, 
                                                                                 output_attentions=True,
                                                                                 output_hidden_states=True))
        
        self.out_features = out_features#model.encoder.layer[1].output.dense.out_features
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.out_features, num_labels) # load and initialize weights

    def forward(self, input_ids=None, attention_mask=None,labels=None):
        #Extract outputs from the body
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)

        #Add custom layers
        sequence_output = self.dropout(outputs[0]) #outputs[0]=last hidden state
        
        logits = self.classifier(sequence_output[:,0,:].view(-1,self.out_features)) # calculate losses

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()

            loss = loss_fct(logits.view(-1, self.num_labels), labels.float().view(-1, self.num_labels))

        return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states,attentions=outputs.attentions)

device = torch.device("cuda" if torch.cuda.is_available() else "mps")
model = CustomModel(checkpoint=checkpoint, num_labels=n_classes, out_features=out_features).to(device)

from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_dataset["train"], shuffle=True, batch_size=BATCH_SIZE, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_dataset["valid"], batch_size=BATCH_SIZE, collate_fn=data_collator
)

from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

from tqdm.auto import tqdm

metric = load_metric("accuracy")

progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(num_epochs * len(eval_dataloader)))


for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar_train.update(1)

    model.eval()
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        targets = torch.argmax(batch["labels"], dim=-1)
        metric.add_batch(predictions=predictions, references=targets)
        progress_bar_eval.update(1)

    print(metric.compute())


torch.save(model, 'bert_model_v1.pt')

metric = load_metric("accuracy")

model.eval()

test_dataloader = DataLoader(
    tokenized_dataset["test"], batch_size=BATCH_SIZE, collate_fn=data_collator
)

for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits

    predictions = torch.argmax(logits, dim=-1)
    targets = torch.argmax(batch["labels"], dim=-1)
    metric.add_batch(predictions=predictions, references=targets)

print(metric.compute())

Map:   0%|          | 0/2219 [00:00<?, ? examples/s]

Map:   0%|          | 0/476 [00:00<?, ? examples/s]

Map:   0%|          | 0/476 [00:00<?, ? examples/s]

Some weights of the model checkpoint at DeepPavlov/distilrubert-small-cased-conversational were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/Users/rvsl/anaconda3/envs/torch-gpu/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.A

  0%|          | 0/834 [00:00<?, ?it/s]

  0%|          | 0/180 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'accuracy': 0.5105042016806722}
{'accuracy': 0.5336134453781513}
{'accuracy': 0.5378151260504201}
{'accuracy': 0.5441176470588235}


In [50]:
metric = load_metric("recall")

model.eval()

test_dataloader = DataLoader(
    tokenized_dataset["test"], batch_size=BATCH_SIZE, collate_fn=data_collator
)

preds = []
labs = []
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits

    predictions = torch.argmax(logits, dim=-1)
    targets = torch.argmax(batch["labels"], dim=-1)
    metric.add_batch(predictions=predictions, references=targets)
    
    preds += list(torch.argmax(logits, dim=-1).detach().cpu().numpy())
    labs += list(torch.argmax(batch["labels"], dim=-1).detach().cpu().numpy())

print(metric.compute(average='macro'))

test = tokenized_dataset["test"].to_pandas()
test['preds'] = preds
test['true'] = labs

from sklearn.metrics import classification_report

print(classification_report(test['true'], test['preds']))

{'recall': 0.5873077586317496}
              precision    recall  f1-score   support

           0       0.84      0.58      0.69        55
           1       0.85      0.63      0.72        27
           2       0.94      0.45      0.61        33
           3       0.52      0.41      0.46        68
           4       0.83      0.65      0.73        31
           5       0.75      0.59      0.66        46
           6       0.57      0.54      0.55        52
           7       0.47      0.45      0.46        49
           8       0.57      0.79      0.67        34
           9       1.00      0.39      0.57        33
          10       0.13      0.60      0.21        15
          11       0.33      0.18      0.24        11
          12       0.13      0.80      0.23         5
          13       0.45      0.83      0.59         6
          14       0.62      0.91      0.74        11

    accuracy                           0.54       476
   macro avg       0.60      0.59      0.54      

In [66]:
# metric = load_metric("recall")

model.eval()

test_dataloader = DataLoader(
    tokenized_dataset["test"], batch_size=BATCH_SIZE, collate_fn=data_collator
)

preds = []
labs = []
thresh = 0.85
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    l = torch.sigmoid(logits).cpu().numpy().tolist()
    
    predictions = [[1 if i > thresh else 0 for i in j] for j in l]
#     break

#     predictions = torch.argmax(logits, dim=-1)
#     targets = torch.argmax(batch["labels"], dim=-1)
    targets = batch["labels"].cpu().numpy().tolist()
    preds.extend(predictions)
    labs.extend(targets)
    
    
#     preds += list(torch.argmax(logits, dim=-1).detach().cpu().numpy())
#     labs += list(torch.argmax(batch["labels"], dim=-1).detach().cpu().numpy())


test = tokenized_dataset["test"].to_pandas()
test['preds'] = preds
test['true'] = labs

from sklearn.metrics import classification_report

# print(classification_report(test['true'], test['preds']))

In [67]:
test.head()

,headline,label,__index_level_0__,input_ids,attention_mask,preds,true
0,"Самое ужасное что только было, не стоит своих ...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1896,"[101, 5419, 34423, 825, 1031, 1035, 128, 802, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,Не полностью проработана система информативнос...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, ...",2044,"[101, 1067, 3161, 41662, 814, 3679, 2269, 8087...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, ..."
2,"Самое убогое приложение, котороя я видел. Раз ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2410,"[101, 5419, 80236, 17589, 128, 1808, 359, 358,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"Навязывают оставить отзыв и поставить оценку, ...","[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1493,"[101, 16528, 10324, 1466, 7529, 18358, 322, 57...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"Обновление ужасное, все лагает, некоторые пози...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",778,"[101, 85227, 34423, 128, 888, 12598, 303, 798,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [77]:
true0 = test['true'].values[0]
pred0 = test['preds'].values[0]

In [81]:
def pos_recall(true0, pred0):
    r = 0
    for i, v in enumerate(true0):
        if pred0[i] == v : r += 1
        else: r -+ 1
    return r/len(true0)

In [85]:
res = []
for true0, pred0 in zip(test['true'].values, test['preds'].values):
    res.append(pos_recall(true0, pred0))
np.mean(np.array(res))

0.9100840336134455